# 1D Motion

### **Introduction**

We live in a quite complex world, where motion is considered a synonym of life. In fact, everything we do is related to motion. 

Our world is composed of 3 dimensions (3D) in which you can move (jump, run, swim, do backflips, etc.). In this lab, we will explore only a part of that complexity of the world we live in - Motion in a single dimension. 


### **What is it?**
What is this type of Motion and why should we study it? Well first, 1D motion is a motion of an object in a straight line. The reason to study this physics topic is to understand the topic of motion. As was mentioned previously, our world comprises 3 dimensions. Why do we start from only one? The thing is, even 1D motion is able to tell us useful information and help to derive certain physical and mathematical principles. 


### **Example**
Electric supercar manufacturer Rimac claimed the acceleration record from a still position hitting 400 km/h and dropping down to 0 in 29.93 seconds. Later on, the engineers of Koenigsegg (probably) took that record personally and made a car that can do 0-400-0 km/h in just 27.83 seconds.

The interesting moment is how the cars were moving when they were beating one record after another. They were mimicking the trajectory of a line.

Although as was said many times previously, our world is 3 dimensional. While we think the car moves in a line, it is also subjected to the gravitation that acts on objects in the vertical dimension (perpendicular to a car). So we can't really do a 1D motion on our planet without utilizing special instruments. In simulations nevertheless, we can enable the dimensions we want.

### **What types of motion exist?**

There are:

- Uniform Motion
- Non-Uniform Motion

Here, we will study the first type of motion, which consists of:

- Uniform Motion
- Uniformly Accelerated Motion
- Uniformly Decelerated Motion

The figure below shows those types of motion:

<img src="../../media/Linear Motion.png" />

### **Formulas**

Motion might be calculated using the following formulas:

Velocity: $\vec{v}=\vec{v_0}+\vec{a}t$\
Position: $x=x_0+\vec{v}t+\frac{1}{2}\vec{a}t^2$

Velocity and position have an interesting realtion. We may derive velocity by taking the derivative from position and derive position by integrating the velocity (which you could learn at Calculus):

* $\vec{v} =\frac{dx}{dt}$
* $x = \int{\vec{v}}dt$ 

# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab
!git clone https://github.com/commanderxa/alphalabs.git

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

# simulation
from dm_control import mjcf

# for video recording
import mediapy

# computations
import numpy as np

# plot charts
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

**Note**, don't set very high values for velocity as the simulation might crash. If you will experience such a situation, try reducing the velocity!

In [ ]:
# global
viscosity = 0  # Air Resistance

# simulation constants
# inital velocity [m/s]
v = 10
# acceleration [m/s^2]
a = -1

size = 0.3
platform_length = 500

# rendering
width = 1920
height = 1080
dpi = 600
duration = 10  # (seconds)
framerate = 60  # (Hz)

## Model

### Objects of Interest

Here we create all the objects that will interact in the world. To model `1D motion` one might create a moving box.

In [ ]:
class Box(object):

    def __init__(self, size: float) -> None:
        self.model = mjcf.RootElement(model="box")

        self.box = self.model.worldbody.add("body", name="box", pos=[0, 0, 0])
        self.box_geom = self.box.add(
            "geom",
            name="box_geom",
            type="box",
            size=[size * 2, size, size],
            mass=1,
            rgba=[1, 0, 0, 1],
            condim=1,
        )

        self.camera = self.box.add(
            "camera",
            name="front",
            pos=[0, -4, 1],
            euler=[75, 0, 0],
            mode="track",
            fovy=45,
        )

        self.move = self.box.add("joint", name="move", type="slide", axis=[1, 0, 0])
        self.fall = self.box.add("joint", name="fall", type="slide", axis=[0, 0, 1])

### World Model

Collecting everything into one general model: Scene and Object. Here, we also may set certain attributes for the MuJoCo engine, for example `viscosity` and `integrator`. 

In [ ]:
class Model(object):

    def __init__(self, size: float, platform_length: float) -> None:
        self.model = mjcf.RootElement(model="model")

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width
        self.model.visual.map.znear = 0.001

        # set the simulation constants
        self.model.option.viscosity = viscosity
        self.model.option.integrator = "RK4"
        self.model.option.timestep = 0.001

        # create the scene (ground)
        self.scene = Scene(length=platform_length, width=2)
        self.scene_site = self.model.worldbody.add(
            "site", pos=[platform_length / 2 - size * 2, 0, 0]
        )
        self.scene_site.attach(self.scene.model)

        # add the object
        self.box = Box(size)
        box_site = self.model.worldbody.add("site", pos=[0, 0, size / 2 + 0.1])
        box_site.attach(self.box.model)

## Simulation

Initializing the `physics` of the simulation

In [ ]:
model = Model(size, platform_length).model
physics = mjcf.Physics.from_mjcf_model(model)

First of all, the environment must be verified by rendering a picture

In [ ]:
mediapy.show_image(physics.render(height=320, width=640, camera_id=0))

### Simulation Loop

In [ ]:
frames = []
timevals = []
velocity = []
position = []

physics.named.data.qvel["box/move"] = v

while physics.data.time < duration:
    
    # applying the acceleration specifier in `a`
    physics.named.data.qvel["box/move"] += a * physics.model.opt.timestep

    # we can obtain certain properties of the objects and thw world,
    # here we writing the time of the step, velocity and position of the `box`
    timevals.append(physics.data.time)
    velocity.append(physics.named.data.qvel["box/move"][0].copy())
    position.append(physics.named.data.geom_xpos["box/box_geom"][0].copy())

    if len(frames) < physics.data.time * framerate:
        pixels = physics.render(width=width, height=height, camera_id=0)
        frames.append(pixels)
    
    physics.step()

In [ ]:
# Simulate and display video.
mediapy.show_video(frames, fps=framerate)

Save the rendered video

In [ ]:
m_type = "acceleration" if a > 0 else "deceleration"
if a == 0:
    m_type = "constant"

video_name = f"1d_motion_{m_type}.mp4" if IS_COLAB else f"../../output/1d_motion.mp4"
mediapy.write_video(video_name, images=frames, fps=framerate)

## Data Analysis

Collected velocities and position now can be plotted to investigate what happened to the object.

Convert data into numpy array to have more features

In [ ]:
velocity = np.array(velocity)
position = np.array(position)

In [ ]:
cm = 1 / 2.54
figsize = (8.5 * cm, 3.75 * cm)
fig, ax = plt.subplots(ncols=2, figsize=figsize, dpi=dpi)
fig.subplots_adjust(wspace=1 * cm)

y_labels = ["Amplitude [m/s]", "Range, [m]"]
x_labels = ["Time, [s]", "Time, [s]"]
titles = ["Velocity", "Position"]
data = [velocity, position]

for i in range(2):
    sns.lineplot(x=timevals, y=data[i], ax=ax[i], linewidth=1, color="red")
    ax[i].set_title(titles[i], fontsize=8)
    ax[i].set_ylabel(y_labels[i], fontsize=7, labelpad=2)
    ax[i].set_xlabel(x_labels[i], fontsize=7, labelpad=2)
    ax[i].tick_params(labelsize=6)
    ax[i].tick_params(which="both", direction="in", top=True, right=True, length=2)
    ax[i].xaxis.set_major_locator(ticker.MaxNLocator(nbins=6))
    ax[i].yaxis.set_major_locator(ticker.MaxNLocator(nbins=6))

chart_name = f"1d_motion_{m_type}.png" if IS_COLAB else f"../../output/1d_motion.png"
fig.savefig(chart_name, bbox_inches="tight")